[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openlayer-ai/examples-gallery/blob/main/tabular-classification/documentation-tutorial/tabular-tutorial-part-4.ipynb)

# <a id="top">Openlayer tabular tutorial - Part 4</a>

Welcome! This is the final notebook from the tabular tutorial. Here, we solve the **performance** issues and commit the new datasets and model versions to the platform. You should use this notebook together with the **tabular tutorial from our documentation**.



## <a id="toc">Table of contents</a>

1. [**Fixing the subpopulation issue and re-training the model**](#1)
    

2. [**Using Openlayer's Python API**](#2)

In [ ]:
%%bash

if [ ! -e "requirements.txt" ]; then
    curl "https://raw.githubusercontent.com/openlayer-ai/examples-gallery/main/tabular-classification/documentation-tutorial/requirements.txt" --output "requirements.txt"
fi

In [ ]:
!pip install -r requirements.txt

## <a id="1"> 1. Fixing the data integrity issues and re-training the model </a>

[Back to top](#top)

In this first part, we will fix the identified data integrity issues in the training and validation sets and re-train the model.   

In [ ]:
import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

### <a id="download">Downloading the dataset </a>

First, we download the same data we used in the previous part of the tutorial, i.e., the data without integrity or consistency issues:

In [ ]:
%%bash

if [ ! -e "churn_train_consistency_fix.csv" ]; then
    curl "https://openlayer-static-assets.s3.us-west-2.amazonaws.com/examples-datasets/tabular-classification/documentation/churn_train_consistency_fix.csv" --output "churn_train_consistency_fix.csv"
fi

if [ ! -e "churn_val_consistency_fix.csv" ]; then
    curl "https://openlayer-static-assets.s3.us-west-2.amazonaws.com/examples-datasets/tabular-classification/documentation/churn_val_consistency_fix.csv" --output "churn_val_consistency_fix.csv"
fi

In [ ]:
train_df = pd.read_csv("./churn_train_consistency_fix.csv")
val_df = pd.read_csv("./churn_val_consistency_fix.csv")

We have diagnosed that a big issue with our model was due to the fact that the subpopulation we found was underrepresented in the training data. Therefore, let's download some new production data and augment our training set with the exact data we need.

In [ ]:
%%bash

if [ ! -e "production_data.csv" ]; then
    curl "https://openlayer-static-assets.s3.us-west-2.amazonaws.com/examples-datasets/tabular-classification/documentation/production_data.csv" --output "production_data.csv"
fi

In [ ]:
production_data = pd.read_csv("./production_data.csv")

In [ ]:
# Get more data that looks like the subpopulation of interest
subpopulation_data = production_data[
    (production_data["Gender"] == "Female") & 
    (production_data["Age"] < 41.5) & 
     (production_data["NumOfProducts"] < 1.5)
]

In [ ]:
train_df = pd.concat([train_df, subpopulation_data], axis=0, ignore_index=True)

In [ ]:
feature_names = [
    "CreditScore", 
    "Geography",
    "Gender",
    "Age", 
    "Tenure",
    "Balance",
    "NumOfProducts",
    "HasCrCard",
    "IsActiveMember",
    "EstimatedSalary"
]
label_column_name = "Exited"

x_train = train_df[feature_names]
y_train = train_df[label_column_name]

x_val = val_df[feature_names]
y_val = val_df[label_column_name]

### <a id="prepare">Preparing the data</a>

In [ ]:
def data_encode_one_hot(df, encoders):
    """ Encodes categorical features using one-hot encoding. """
    df = df.copy(True)
    df.reset_index(drop=True, inplace=True) # Causes NaNs otherwise
    for feature, enc in encoders.items():
        enc_df = pd.DataFrame(enc.transform(df[[feature]]).toarray(), columns=enc.get_feature_names([feature]))
        df = df.join(enc_df)
        df = df.drop(columns=feature)
    return df

In [ ]:
def create_encoder_dict(df, categorical_feature_names):
    """ Creates encoders for each of the categorical features. 
        The predict function will need these encoders. 
    """
    from sklearn.preprocessing import OneHotEncoder
    encoders = {}
    for feature in categorical_feature_names:
        enc = OneHotEncoder(handle_unknown='ignore')
        enc.fit(df[[feature]])
        encoders[feature] = enc
    return encoders

In [ ]:
encoders = create_encoder_dict(x_train, ['Geography', 'Gender'])

In [ ]:
x_train_one_hot = data_encode_one_hot(x_train, encoders)
x_val_one_hot = data_encode_one_hot(x_val, encoders)

### <a id="train">Training the model</a>

In [ ]:
sklearn_model = GradientBoostingClassifier(random_state=1300)
sklearn_model.fit(x_train_one_hot, y_train)

In [ ]:
print(classification_report(y_val, sklearn_model.predict(x_val_one_hot)))

## <a id="2"> 2. Using Openlayer's Python API</a>

[Back to top](#top)

Now it's time to upload the datasets and model to the Openlayer platform.

In [ ]:
!pip install openlayer

### <a id="client">Instantiating the client</a>

In [ ]:
import openlayer

client = openlayer.OpenlayerClient("YOUR_API_KEY_HERE")

### <a id="project">Creating a project on the platform</a>

In [ ]:
from openlayer.tasks import TaskType

project = client.create_or_load_project(
    name="Churn Prediction",
    task_type=TaskType.TabularClassification,
    description="Evaluation of ML approaches to predict churn"
)

### <a id="dataset">Uploading datasets</a>

In [ ]:
# Adding the column with the labels
training_set = x_train.copy(deep=True)
training_set["Exited"] = y_train.values
validation_set = x_val.copy(deep=True)
validation_set["Exited"] = y_val.values

In [ ]:
# Adding the column with the predictions (since we'll also upload a model later)
training_set["predictions"] = sklearn_model.predict_proba(x_train_one_hot).tolist()
validation_set["predictions"] = sklearn_model.predict_proba(x_val_one_hot).tolist()

Now, we can prepare the `dataset_config.yaml` files for the training and validation sets.

In [ ]:
# Some variables that will go into the `dataset_config.yaml` file
categorical_feature_names = ["Gender", "Geography"]
class_names = ["Retained", "Exited"]
column_names = list(training_set.columns)
feature_names = list(x_val.columns)
label_column_name = "Exited"
predictions_column_name = "predictions"

In [ ]:
import yaml 

# Note the camelCase for the dict's keys
training_dataset_config = {
    "categoricalFeatureNames": categorical_feature_names,
    "classNames": class_names,
    "columnNames": column_names,
    "featureNames":feature_names,
    "label": "training",
    "labelColumnName": label_column_name,
    "predictionsColumnName": predictions_column_name,
}

with open("training_dataset_config.yaml", "w") as dataset_config_file:
    yaml.dump(training_dataset_config, dataset_config_file, default_flow_style=False)

In [ ]:
import copy

validation_dataset_config = copy.deepcopy(training_dataset_config)

# In our case, the only field that changes is the `label`, from "training" -> "validation"
validation_dataset_config["label"] = "validation"

with open("validation_dataset_config.yaml", "w") as dataset_config_file:
    yaml.dump(validation_dataset_config, dataset_config_file, default_flow_style=False)

In [ ]:
# Training set
project.add_dataframe(
    dataset_df=training_set,
    dataset_config_file_path="training_dataset_config.yaml",
)

In [ ]:
# Validation set
project.add_dataframe(
    dataset_df=validation_set,
    dataset_config_file_path="validation_dataset_config.yaml",
)

We can check that both datasets are now staged using the `project.status()` method. 

In [ ]:
project.status()

### <a id="model">Uploading models</a>

Again, we will upload a full model. Considering the model package we prepared in the previous notebook, the only component that needs to be changed is the serialized artifacts. The remaining components (i.e., the requirements file, the `prediction_interface.py`, and model config) remain the same.

If you already have the `model_package` locally, feel free to update just the artifacts. In the next few cells we re-create the model package so that this notebook is self-contained.

In [ ]:
# Creating the model package folder (we'll call it `model_package`)
!mkdir model_package

In [ ]:
!scp requirements.txt model_package

In [ ]:
import pickle 

# Trained model
with open("model_package/model.pkl", "wb") as handle:
    pickle.dump(sklearn_model, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Encoder for the categorical features
with open("model_package/encoders.pkl", "wb") as handle:
    pickle.dump(encoders, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
%%writefile model_package/prediction_interface.py

import pickle
from pathlib import Path

import pandas as pd
from sklearn.preprocessing import OneHotEncoder

PACKAGE_PATH = Path(__file__).parent


class SklearnModel:
    def __init__(self):
        """This is where the serialized objects needed should
        be loaded as class attributes."""

        with open(PACKAGE_PATH / "model.pkl", "rb") as model_file:
            self.model = pickle.load(model_file)
        with open(PACKAGE_PATH / "encoders.pkl", "rb") as encoders_file:
            self.encoders = pickle.load(encoders_file)

    def _data_encode_one_hot(self, df: pd.DataFrame) -> pd.DataFrame:
        """Pre-processing needed for our particular use case."""

        df = df.copy(True)
        df.reset_index(drop=True, inplace=True)  # Causes NaNs otherwise
        for feature, enc in self.encoders.items():
            enc_df = pd.DataFrame(
                enc.transform(df[[feature]]).toarray(),
                columns=enc.get_feature_names([feature]),
            )
            df = df.join(enc_df)
            df = df.drop(columns=feature)
        return df

    def predict_proba(self, input_data_df: pd.DataFrame):
        """Makes predictions with the model. Returns the class probabilities."""

        encoded_df = self._data_encode_one_hot(input_data_df)
        return self.model.predict_proba(encoded_df)


def load_model():
    """Function that returns the wrapped model object."""
    return SklearnModel()

In [ ]:
import yaml

model_config = {
    "name": "Churn classifier",
    "architectureType": "sklearn",
    "metadata": {  # Can add anything here, as long as it is a dict
        "model_type": "Gradient Boosting Classifier",
        "regularization": "None",
        "encoder_used": "One Hot",
    },
    "classNames": class_names,
    "featureNames": feature_names,
    "categoricalFeatureNames": categorical_feature_names,
}

with open("model_config.yaml", "w") as model_config_file:
    yaml.dump(model_config, model_config_file, default_flow_style=False)

In [ ]:
project.add_model(
    model_package_dir="model_package",
    model_config_file_path="model_config.yaml",
    sample_data=x_val.iloc[:10, :],
)

We can check that both datasets and model are staged using the `project.status()` method.

In [ ]:
project.status()

### <a id="commit"> Committing and pushing to the platform </a>

Finally, we can commit the first project version to the platform. 

In [ ]:
project.commit("Fixes subpopulation issue")

In [ ]:
project.status()

In [ ]:
project.push()